In [2]:
import requests
import json
import pandas as pd
from datetime import datetime

In [3]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
    'Content-Type': 'application/json'}
url = 'https://www.dailyhotel.com/newdelhi/goodnight/api/v1/stay/region?regionStayType=new_all'

In [4]:
res = requests.get(url,headers=headers)
region = json.loads(res.text)

In [5]:
region_list = []
for i in region['data']:

    
    region_list.append({
        'idx':i['idx'],
        'name':i['name'],
        'regions':i['regions']
    })

region_list

[{'idx': 300000016,
  'name': '서울',
  'regions': [{'idx': 200000151, 'name': '강남 / 역삼 / 삼성', 'regions': []},
   {'idx': 200000152, 'name': '서초 / 교대 / 방배', 'regions': []},
   {'idx': 200000153, 'name': '사당', 'regions': []},
   {'idx': 200000154, 'name': '신사 / 압구정 / 청담', 'regions': []},
   {'idx': 200000155, 'name': '잠실 / 송파 / 잠실새내 / 천호 / 강동', 'regions': []},
   {'idx': 200000156, 'name': '서울역 / 이태원 / 용산', 'regions': []},
   {'idx': 200000157, 'name': '명동 / 중구', 'regions': []},
   {'idx': 200000158, 'name': '을지로 / 충무로', 'regions': []},
   {'idx': 200000159, 'name': '종로 / 동대문 / 대학로', 'regions': []},
   {'idx': 200000160, 'name': '인사동', 'regions': []},
   {'idx': 200000161, 'name': '서대문 / 마포 / 신촌 / 홍대', 'regions': []},
   {'idx': 200000162, 'name': '영등포', 'regions': []},
   {'idx': 200000163, 'name': '여의도', 'regions': []},
   {'idx': 200000164, 'name': '신림 / 서울대', 'regions': []},
   {'idx': 200000165, 'name': '구로 / 금천', 'regions': []},
   {'idx': 200000166, 'name': '김포공항', 'regions': []},


In [63]:
region_list[1]['idx'], region_list[1]['name']

(300000017, '부산')

In [64]:
region_list[1]['regions']

[{'idx': 200000173, 'name': '해운대', 'regions': []},
 {'idx': 200000174, 'name': '광안리 / 센텀', 'regions': []},
 {'idx': 200000175, 'name': '경성대', 'regions': []},
 {'idx': 200000176, 'name': '송정 / 기장', 'regions': []},
 {'idx': 200000177, 'name': '부산역', 'regions': []},
 {'idx': 200000178, 'name': '남포 / 자갈치', 'regions': []},
 {'idx': 200000179, 'name': '송도', 'regions': []},
 {'idx': 200000180, 'name': '서면 / 양정', 'regions': []},
 {'idx': 200000181, 'name': '동래', 'regions': []},
 {'idx': 200000182, 'name': '연산', 'regions': []},
 {'idx': 200000183, 'name': '금정', 'regions': []},
 {'idx': 200000184, 'name': '사상', 'regions': []},
 {'idx': 200000185, 'name': '명지', 'regions': []},
 {'idx': 200000186, 'name': '화명', 'regions': []},
 {'idx': 200000187, 'name': '신호 / 하단', 'regions': []},
 {'idx': 300000017, 'name': '부산 전체', 'regions': []}]

In [47]:
# detail['data']['staySaleSections'][0]['sectionType']
import time

In [65]:
today = datetime.now().strftime('%Y-%m-%d')
total_df = pd.DataFrame()

for region in region_list:
    r_idx = region['idx']
    r_name = region['name']

    region_df = pd.DataFrame()

    for city in region['regions']:
        

        r_hotels = []
        r_num_check = 0

        c_idx = city['idx']
        c_name = city['name']

        if '전체' not in c_name:
            print(c_idx, c_name)
            
            url = f'https://www.dailyhotel.com/newdelhi/goodnight/api/v10/stay/plp/list?dateCheckIn={today}&regionIdx={c_idx}&shortCutType=stayAll&persons=1&stays=1&details=true&regionStayType=new_all&stayType=&page={page}&limit=50'
            res = requests.get(url, headers=headers)
            detail = json.loads(res.text)
            page = (detail['data']['staySalesCount'] // 50) + 2
                
            for i in range(1,page):

                url = f'https://www.dailyhotel.com/newdelhi/goodnight/api/v10/stay/plp/list?dateCheckIn={today}&regionIdx={c_idx}&shortCutType=stayAll&persons=1&stays=1&details=true&regionStayType=new_all&stayType=&page={page}&limit=50'
                res = requests.get(url, headers=headers)
                detail = json.loads(res.text)

                r_num_check += detail['data']['staySalesCount']

                for section in detail['data']['staySaleSections']:
                    # 광고 스킵
                    if section['sectionType'] != 'NORMAL':
                        continue

                    for hotel in section['staySales']:
                        r_hotels.append({
                            'code': hotel['hotelIdx'],
                            'name': hotel['name'],
                            'grade': hotel['stayGradeName'],
                            'type': hotel['stayTypeCode'],
                            'd_price': hotel['discount'],
                            'price': hotel['price'],
                            'benefit': hotel['benefit'],
                            'region_L': hotel['regionLargeName'],
                            'region_S': hotel['regionMediumName'],
                            'code_L': hotel['regionLargeIdx'],
                            'code_S': hotel['regionMediumIdx'],
                            'lat': hotel['latitude'],
                            'lon': hotel['longitude']
                            })
                        time.sleep(0.06)
                
                
            print(page)
                
            df = pd.DataFrame(r_hotels).drop_duplicates().reset_index()

            if len(df) != r_num_check:
                print(f'hotel number error, {c_name}: {len(df)}, {r_num_check}')
                break

        region_df = pd.concat([region_df, df], ignore_index=True)

    region_df.to_csv(f'./hotels/{r_name}.csv')
    total_df = pd.concat([total_df, region_df], ignore_index=True)

total_df.to_csv('./hotels/전체.csv')

200000151 강남 / 역삼 / 삼성
3
hotel number error, 강남 / 역삼 / 삼성: 0, 176
200000173 해운대
5
hotel number error, 해운대: 0, 620
200000188 제주시


KeyboardInterrupt: 

In [47]:
# 찐

today = datetime.now().strftime('%Y-%m-%d')
total_df = pd.DataFrame()

for region in region_list:
    r_idx = region['idx']
    r_name = region['name']

    region_df = pd.DataFrame()

    for city in region['regions']:
        
        page = 1
        max_page = 10
        r_hotels = []
        r_num_check = 0

        c_idx = city['idx']
        c_name = city['name']

        if '전체' not in c_name:
            print(c_idx, c_name)
            
            while page <= max_page:
                url = f'https://www.dailyhotel.com/newdelhi/goodnight/api/v10/stay/plp/list?dateCheckIn={today}&regionIdx={c_idx}&shortCutType=stayAll&persons=1&stays=1&details=true&regionStayType=new_all&stayType=&page={page}&limit=50'
                res = requests.get(url, headers=headers)
                detail = json.loads(res.text)
                
                # if page == 1:
                #     r_num_check += detail['data']['staySalesCount']

                for section in detail['data']['staySaleSections']:
                    # 광고 스킵
                    if section['sectionType'] != 'NORMAL':
                        continue

                    for hotel in section['staySales']:
                        r_hotels.append({
                            'code': hotel['hotelIdx'],
                            'name': hotel['name'],
                            'grade': hotel['stayGradeName'],
                            'type': hotel['stayTypeCode'],
                            'd_price': hotel['discount'],
                            'price': hotel['price'],
                            'benefit': hotel['benefit'],
                            'region_L': hotel['regionLargeName'],
                            'region_S': hotel['regionMediumName'],
                            'code_L': hotel['regionLargeIdx'],
                            'code_S': hotel['regionMediumIdx'],
                            'lat': hotel['latitude'],
                            'lon': hotel['longitude']
                            })
                max_page = (detail['data']['staySalesCount'] // 50) + 1
                page += 1
            print(page)
                
            df = pd.DataFrame(r_hotels).drop_duplicates().reset_index()
            r_num_check += detail['data']['staySalesCount']
            
            if len(df) != r_num_check:
                print(f'hotel number error, {c_name}: {len(df)}, {r_num_check}')
                break

        region_df = pd.concat([region_df, df], ignore_index=True)

    region_df.to_csv(f'./hotels/{r_name}.csv')
    total_df = pd.concat([total_df, region_df], ignore_index=True)

total_df.to_csv('./hotels/전체.csv')

200000151 강남 / 역삼 / 삼성
3
200000152 서초 / 교대 / 방배
2
200000153 사당
2
200000154 신사 / 압구정 / 청담
2
200000155 잠실 / 송파 / 잠실새내 / 천호 / 강동
4
200000156 서울역 / 이태원 / 용산
2
200000157 명동 / 중구
5
200000158 을지로 / 충무로
2
200000159 종로 / 동대문 / 대학로
2
200000160 인사동
3
200000161 서대문 / 마포 / 신촌 / 홍대
4
200000162 영등포
3
200000163 여의도
2
200000164 신림 / 서울대
2
200000165 구로 / 금천
3
200000166 김포공항
2
200000167 강서 / 화곡
3
200000168 건대 / 성수
3
200000169 왕십리
2
200000170 군자 / 상봉 / 중랑구
2
200000171 장안 / 청량리
2
200000172 강북 / 수유 / 도봉 / 미아 / 성북 / 노원
4
200000173 해운대
5
hotel number error, 해운대: 154, 155
200000188 제주시
10
hotel number error, 제주시: 406, 419
200000198 강릉 / 경포대 / 정동진
15
hotel number error, 강릉 / 경포대 / 정동진: 623, 663
200000212 수원 / 인계 / 권선 / 영통
4
200000213 장안 / 팔달
3
200000214 용인
5
hotel number error, 용인: 163, 165
200000234 강화


KeyboardInterrupt: 

In [ ]:
pd.set_option('display.max_rows', 15)

In [34]:
r_hotels = []
for i in range(1,5):
    url = f'https://www.dailyhotel.com/newdelhi/goodnight/api/v10/stay/plp/list?dateCheckIn=2023-11-06&regionIdx=200000157&shortCutType=stayAll&persons=1&stays=1&details=true&regionStayType=new_all&stayType=&page={i}&limit=50'

    res = requests.get(url, headers=headers)
    detail = json.loads(res.text)

    for section in detail['data']['staySaleSections']:
        for hotel in section['staySales']:
            r_hotels.append({
                'code': hotel['hotelIdx'],
                'name': hotel['name'],
                'grade': hotel['stayGradeName'],
                'type': hotel['stayTypeCode'],
                'd_price': hotel['discount'],
                'price': hotel['price'],
                'benefit': hotel['benefit'],
                'region_L': hotel['regionLargeName'],
                'region_S': hotel['regionMediumName'],
                'code_L': hotel['regionLargeIdx'],
                'code_S': hotel['regionMediumIdx'],
                'lat': hotel['latitude'],
                'lon': hotel['longitude']
                })

b = pd.DataFrame(r_hotels)
b


,code,name,grade,type,d_price,price,benefit,region_L,region_S,code_L,code_S,lat,lon
0,84970,동대문역사문화공원역 LOO(루호텔),모텔,motel,0,0,GRAND OPEN 디즈니+/넷플릭스 무료,서울,명동 / 중구,300000016,200000157,37.565362,127.007024
1,79148,동대문역사문화공원역 부티크호텔 르릿,모텔,motel,0,0,매일 소독넷플릭스셀프체크인,서울,명동 / 중구,300000016,200000157,37.563459,127.008529
2,70052,프레이저 플레이스 센트럴,4성급,hotel,209000,209000,,서울,명동 / 중구,300000016,200000157,37.562782,126.969783
3,75413,노보텔 앰배서더 동대문 호텔 & 레지던스,5성급,hotel,281600,550000,,서울,명동 / 중구,300000016,200000157,37.566159,127.004259
4,84589,서울신라호텔,5성급,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.555881,127.005153
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,80814,메트로 호텔,호텔,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.565096,126.983716
150,88797,명동 사보이 호텔,호텔,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.561766,126.984276
151,73159,호텔 릿,호텔,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.554624,126.973805
152,79443,명동 호텔미도,모텔,motel,0,0,,서울,명동 / 중구,300000016,200000157,37.558416,126.980561


In [35]:
len(b)

154

In [36]:
detail['data']['staySalesCount']

151

In [37]:
len(b.drop_duplicates().reset_index())

150

In [39]:
comparison = a == b

In [40]:
comparison.

,code,name,grade,type,d_price,price,benefit,region_L,region_S,code_L,code_S,lat,lon
0,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,True,True,True,True,True,True,True,True,True,True,True,True,True
150,True,True,True,True,True,True,True,True,True,True,True,True,True
151,False,False,False,False,True,True,True,True,True,True,True,False,False
152,False,False,True,True,True,True,True,True,True,True,True,False,False


In [41]:
print(comparison[comparison == False])

      code   name  grade   type d_price price benefit region_L region_S  \
0      NaN    NaN    NaN    NaN     NaN   NaN     NaN      NaN      NaN   
1      NaN    NaN    NaN    NaN     NaN   NaN     NaN      NaN      NaN   
2      NaN    NaN    NaN    NaN     NaN   NaN     NaN      NaN      NaN   
3      NaN    NaN    NaN    NaN     NaN   NaN     NaN      NaN      NaN   
4      NaN    NaN    NaN    NaN     NaN   NaN     NaN      NaN      NaN   
..     ...    ...    ...    ...     ...   ...     ...      ...      ...   
149    NaN    NaN    NaN    NaN     NaN   NaN     NaN      NaN      NaN   
150    NaN    NaN    NaN    NaN     NaN   NaN     NaN      NaN      NaN   
151  False  False  False  False     NaN   NaN     NaN      NaN      NaN   
152  False  False    NaN    NaN     NaN   NaN     NaN      NaN      NaN   
153    NaN    NaN    NaN    NaN     NaN   NaN     NaN      NaN      NaN   

    code_L code_S    lat    lon  
0      NaN    NaN    NaN    NaN  
1      NaN    NaN    NaN    NaN

In [42]:
a

,code,name,grade,type,d_price,price,benefit,region_L,region_S,code_L,code_S,lat,lon
0,84970,동대문역사문화공원역 LOO(루호텔),모텔,motel,0,0,GRAND OPEN 디즈니+/넷플릭스 무료,서울,명동 / 중구,300000016,200000157,37.565362,127.007024
1,79148,동대문역사문화공원역 부티크호텔 르릿,모텔,motel,0,0,매일 소독넷플릭스셀프체크인,서울,명동 / 중구,300000016,200000157,37.563459,127.008529
2,70052,프레이저 플레이스 센트럴,4성급,hotel,209000,209000,,서울,명동 / 중구,300000016,200000157,37.562782,126.969783
3,75413,노보텔 앰배서더 동대문 호텔 & 레지던스,5성급,hotel,281600,550000,,서울,명동 / 중구,300000016,200000157,37.566159,127.004259
4,84589,서울신라호텔,5성급,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.555881,127.005153
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,80814,메트로 호텔,호텔,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.565096,126.983716
150,88797,명동 사보이 호텔,호텔,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.561766,126.984276
151,79443,명동 호텔미도,모텔,motel,0,0,,서울,명동 / 중구,300000016,200000157,37.558416,126.980561
152,79453,명동 꼬모,모텔,motel,0,0,,서울,명동 / 중구,300000016,200000157,37.560381,126.987196


In [ ]:
b

In [33]:
a.drop_duplicates().reset_index()

,index,code,name,grade,type,d_price,price,benefit,region_L,region_S,code_L,code_S,lat,lon
0,0,84970,동대문역사문화공원역 LOO(루호텔),모텔,motel,0,0,GRAND OPEN 디즈니+/넷플릭스 무료,서울,명동 / 중구,300000016,200000157,37.565362,127.007024
1,1,79148,동대문역사문화공원역 부티크호텔 르릿,모텔,motel,0,0,매일 소독넷플릭스셀프체크인,서울,명동 / 중구,300000016,200000157,37.563459,127.008529
2,2,70052,프레이저 플레이스 센트럴,4성급,hotel,209000,209000,,서울,명동 / 중구,300000016,200000157,37.562782,126.969783
3,3,75413,노보텔 앰배서더 동대문 호텔 & 레지던스,5성급,hotel,281600,550000,,서울,명동 / 중구,300000016,200000157,37.566159,127.004259
4,4,84589,서울신라호텔,5성급,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.555881,127.005153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,149,80814,메트로 호텔,호텔,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.565096,126.983716
147,150,88797,명동 사보이 호텔,호텔,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.561766,126.984276
148,151,79443,명동 호텔미도,모텔,motel,0,0,,서울,명동 / 중구,300000016,200000157,37.558416,126.980561
149,152,79453,명동 꼬모,모텔,motel,0,0,,서울,명동 / 중구,300000016,200000157,37.560381,126.987196


In [ ]:
a

In [ ]:
a.drop_duplicates().reset_index()

In [ ]:
a.drop_duplicates().reset_index()

In [ ]:
a.to_csv('test.csv')

In [ ]:
'code': hotel['hotelIdx']
'name': hotel['name']
'grade': hotel['stayGradeName']
'type': hotel['stayTypeCode']
'd_price': hotel['discount']
'price': hotel['price']
'benefit': hotel['benefit']
'region_L': hotel['regionLargeName']
'region_S': hotel['regionMediumName']
'code_L': hotel['regionLargeIdx']
'code_S': hotel['regionMediumIdx']
'lat': hotel['latitude']
'lon': hotel['longitude']

In [ ]:
detail['data']['staySaleSections'][0].keys()

In [ ]:
detail['data']['staySaleSections'][1].keys()

In [ ]:
today = datetime.now().strftime('%Y-%m-%d')
total_df = pd.DataFrame()

for region in region_list:
    r_idx = region['idx']
    r_name = region['name']

    region_df = pd.DataFrame()

    for city in region['regions']:
        
        page = 1
        max_page = 10
        r_hotels = []
        r_num_check = 0

        c_idx = city['idx']
        c_name = city['name']

        if '전체' not in c_name:
            print(c_idx, c_name)
            
            while page <= max_page:
                url = f'https://www.dailyhotel.com/newdelhi/goodnight/api/v10/stay/plp/list?dateCheckIn={today}&regionIdx={c_idx}&shortCutType=stayAll&persons=1&stays=1&details=true&regionStayType=new_all&stayType=&page={page}&limit=50'
                res = requests.get(url, headers=headers)
                detail = json.loads(res.text)
                
                if page == 1:
                    r_num_check += detail['data']['staySalesCount']

                for section in detail['data']['staySaleSections']:
                    for hotel in section['staySales']:
                        r_hotels.append({
                            'code': hotel['hotelIdx'],
                            'name': hotel['name'],
                            'grade': hotel['stayGradeName'],
                            'type': hotel['stayTypeCode'],
                            'd_price': hotel['discount'],
                            'price': hotel['price'],
                            'benefit': hotel['benefit'],
                            'region_L': hotel['regionLargeName'],
                            'region_S': hotel['regionMediumName'],
                            'code_L': hotel['regionLargeIdx'],
                            'code_S': hotel['regionMediumIdx'],
                            'lat': hotel['latitude'],
                            'lon': hotel['longitude']
                            })
                max_page = (detail['data']['staySalesCount'] // 50) + 1
                page += 1
        df = pd.DataFrame(r_hotels).drop_duplicates().reset_index()

        if len(df) != r_num_check:
            print(f'hotel number error, {len(df)}, {r_num_check}')
            break
        region_df = pd.concat([region_df, df], ignore_index=True)

    region_df.to_csv(f'./hotels/{r_name}.csv')
    total_df = pd.concat([total_df, region_df], ignore_index=True)

total_df.to_csv('./hotels/전체.csv')

강남 예시 200000151, 300000016
https://www.dailyhotel.com/stays?shortCutType=stayAll&regionStayType=new_all&dateCheckIn=2023-11-06&rai=200000151&rpi=300000016&stays=1&reg=0

In [ ]:

today = datetime.now().strftime('%Y-%m-%d')
total_df = pd.DataFrame()

for region in region_list:
    r_idx = region['idx']
    r_name = region['name']

    region_df = pd.DataFrame()

    for city in region['regions']:
        
        page = 1
        max_page = 10
        r_hotels = []
        r_num_check = 0

        c_idx = city['idx']
        c_name = city['name']

        if '전체' not in c_name:
            print(c_idx, c_name)
            
            while page <= max_page:
                url = f'https://www.dailyhotel.com/newdelhi/goodnight/api/v10/stay/plp/list?dateCheckIn={today}&regionIdx={c_idx}&shortCutType=stayAll&persons=1&stays=1&details=true&regionStayType=new_all&stayType=&page={page}&limit=50'
                res = requests.get(url, headers=headers)
                detail = json.loads(res.text)
                
                if page == 1:
                    r_num_check += detail['data']['staySalesCount']

                for section in detail['data']['staySaleSections']:
                    for hotel in section['staySales']:
                        r_hotels.append({
                            'code': hotel['hotelIdx'],
                            'name': hotel['name'],
                            'grade': hotel['stayGradeName'],
                            'type': hotel['stayTypeCode'],
                            'd_price': hotel['discount'],
                            'price': hotel['price'],
                            'benefit': hotel['benefit'],
                            'region_L': hotel['regionLargeName'],
                            'region_S': hotel['regionMediumName'],
                            'code_L': hotel['regionLargeIdx'],
                            'code_S': hotel['regionMediumIdx'],
                            'lat': hotel['latitude'],
                            'lon': hotel['longitude']
                            })
                max_page = (detail['data']['staySalesCount'] // 50) + 2
                page += 1
        df = pd.DataFrame(r_hotels).drop_duplicates().reset_index()

        if len(df) != r_num_check:
            print(f'hotel number error, {len(df)}, {r_num_check}')
            break
        pd.concat([region_df,df], ignore_index=True)

    region_df.to_csv(f'./hotels/{r_name}.csv')
    pd.concat([total_df,region_df], ignore_index=True)

total_df.to_csv('./hotels/전체.csv')


In [43]:
b

,code,name,grade,type,d_price,price,benefit,region_L,region_S,code_L,code_S,lat,lon
0,84970,동대문역사문화공원역 LOO(루호텔),모텔,motel,0,0,GRAND OPEN 디즈니+/넷플릭스 무료,서울,명동 / 중구,300000016,200000157,37.565362,127.007024
1,79148,동대문역사문화공원역 부티크호텔 르릿,모텔,motel,0,0,매일 소독넷플릭스셀프체크인,서울,명동 / 중구,300000016,200000157,37.563459,127.008529
2,70052,프레이저 플레이스 센트럴,4성급,hotel,209000,209000,,서울,명동 / 중구,300000016,200000157,37.562782,126.969783
3,75413,노보텔 앰배서더 동대문 호텔 & 레지던스,5성급,hotel,281600,550000,,서울,명동 / 중구,300000016,200000157,37.566159,127.004259
4,84589,서울신라호텔,5성급,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.555881,127.005153
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,80814,메트로 호텔,호텔,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.565096,126.983716
150,88797,명동 사보이 호텔,호텔,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.561766,126.984276
151,73159,호텔 릿,호텔,hotel,0,0,,서울,명동 / 중구,300000016,200000157,37.554624,126.973805
152,79443,명동 호텔미도,모텔,motel,0,0,,서울,명동 / 중구,300000016,200000157,37.558416,126.980561
